## Các trạng thái của agent:
- Vị trí tồn kho theo thời gian t. Đối với bài báo, vị trí tồn kho có thể là vô hạn( có thể cho luôn âm vô cực để xét chi tiết thiếu hàng). và nó chỉ xét trên 1 nguyên liệu duy nhất. cho nên trạng thái chỉ cần quan tâm đến vị trí tồn kho(số lượng hàng còn trong kho tính cả hàng đã đặt).
    Với bài làm hiện tại, có đa dạng sản phẩm tồn tại trong kho, cụ thể là có 92353, cho nên việc phân chia trạng thái theo chiều sẽ bất khả thi, vì nó sẽ nhân theo cấp số mũ, thành ra gần như tạo ra vô số trạng thái.
    Giờ đây có một ý tưởng là sẽ áp dụng phần chia sản phẩm theo ABC/XYZ để giảm thiểu đi số lượng loại sản phẩm trong kho đi.
# Cách phân tích ABC/XYZ
- ABC analysis – chia theo giá trị và tầm quan trọng
    - A: ~20% sản phẩm nhưng chiếm ~80% giá trị tồn kho/doanh thu. Đây là nhóm “xương sống” cần quản chặt.

    - B: ~30% sản phẩm, ~15% giá trị. Quản lý vừa phải.

    - C: ~50% sản phẩm, chỉ ~5% giá trị. Quản lý đơn giản hơn.
- XYZ analysis – chia theo tính ổn định nhu cầu
    - X: nhu cầu ổn định, dự đoán tốt (ví dụ bán đều mỗi tháng).

    - Y: nhu cầu có mùa vụ hoặc biến động theo chu kỳ.

    - Z: nhu cầu thất thường, khó dự đoán.
- Kết hợp 2 lại phần tích thành ABC/XYZ thì sẽ có:
    - Mỗi sản phẩm sẽ rơi vào một trong 9 ô (A‑X, A‑Y, A‑Z, B‑X, … C‑Z).

    - Sau đó, có thể gom tiếp các ô có đặc điểm quản lý giống nhau thành khoảng 50 nhóm.

    - Ví dụ: A‑X và B‑X có thể chung nhóm “quản lý chặt và nhu cầu ổn định”.

    - C‑Z có thể thành một nhóm “ít quan trọng, nhu cầu khó đoán”.

Sau khi áp dụng phương pháp phân tích trên, thì từ 92353 loại sản phẩm trong kho thì sẽ chia ra được 50 phân nhóm.

Sau khi chia ra phân nhóm xong, thì tiếp tục hành trình phân tích rời rạc trạng thái. Giờ thì sẽ chia mẫu trạng thái ra làm 4 mức rời rạc (0-3).

Sau khi phân nhóm xong, State = mảng 50 phần tử (mỗi phần tử ∈ {0,1,2,3}) → chỉ còn 
4**50 tổ hợp thay vì với số mũ 92353 tổ hợp ban đầu.

## Phân tích data

In [28]:
import pandas as pd
import os
transaction = pd.read_csv('C:\\Users\\HP ZBook 15\\.cache\\kagglehub\\datasets\\frtgnn\\dunnhumby-the-complete-journey\\versions\\1\\transaction_data.csv')
transaction.head(5)
pd.set_option('display.max_rows', None)
product=pd.read_csv('C:\\Users\\HP ZBook 15\\.cache\\kagglehub\\datasets\\frtgnn\\dunnhumby-the-complete-journey\\versions\\1\\product.csv')
product.tail()
merged_pt=pd.merge(transaction, product,on='PRODUCT_ID')
grouped_pt=merged_pt[['SUB_COMMODITY_DESC','QUANTITY','SALES_VALUE']].groupby('SUB_COMMODITY_DESC').sum().sort_values('QUANTITY',ascending=False)
print(grouped_pt)
print(grouped_pt.count())


                                 QUANTITY  SALES_VALUE
SUB_COMMODITY_DESC                                    
GASOLINE-REG UNLEADED           257331979    633957.26
FLUID MILK WHITE ONLY               91476    161907.17
YOGURT NOT MULTI-PACKS              66198     38471.38
SOFT DRINKS 12/18&15PK CAN CAR      55022    158187.60
SFT DRNK 2 LITER BTL CARB INCL      54432     53602.84
CANDY BARS (SINGLES)(INCLUDING      46285     17777.99
SHREDDED CHEESE                     35245     67662.90
MAINSTREAM WHITE BREAD              34661     44297.34
CONDENSED SOUP                      32955     33514.07
SS ECONOMY ENTREES/DINNERS ALL      31665     31297.39
BANANAS                             31595     30367.28
SOFT DRINK POWDER POUCHES           29291     11192.22
POTATO CHIPS                        24248     53118.01
PASTA: CANNED                       20853     20342.50
PREMIUM                             20574     64758.92
RAMEN NOODLES/RAMEN CUPS            20456      6668.57
BEERALEMAL

## Những phân tích sau khi xem xét 2 data transaction_data và product.
- Sau khi phân tích, có thể thấy rằng khi xét theo SUB_COMMODITY_DESC(loại sản phẩm) thì có tồn tại 1 số loại sản phẩm không bán được sản phẩm nào hoặc bán được dưới 50 sản phẩm trong 711 ngày kể từ khi mở cửa hàng. Từ thế, có thể xét trạng thái theo SUB_COMMODITY_DESC để giảm thiểu khối lượng huấn luyện, có thể loại bỏ huấn luyện một số loại sản phẩm bán ra với số lượng rất ít (<50), có thể coi nó là loại hàng ít phổ biến, chỉ được bán khi khách hàng đặt, không cần trữ sẵn trong kho.

- Với loại mặt hàng GASOLINE-REG UNLEADED( Xăng loại thường không chứa chi ), loại mặt hàng này có nhu cầu mua rất là cao(>10000000), từng sản phẩm của nó cũng có số lượng mua đều >1000 trong 711 ngày. Vì vậy nên, đề xuất tạo thêm 1 agent huấn luyện riêng về mặt hàng này.Có nghĩa là sẽ làm theo hướng multiple-agent để giảm thiểu công việc, không dồn hết 1 công việc huấn luyện vào 1 agent.

- Vì đầy là dataset huấn luyện, nếu làm ứng dụng, thì số sản phẩm sẽ khác đi và số lượng của nó cũng có thể là vô hạn. Cứ mỗi lần thêm 1 sản phẩm thì agent nó lại phải huấn luyện lại từ đầu, nên thiết kế theo hướng multiple-agent sẽ có lợi ích giảm thiểu khối lượng huấn luyện lại, không cần huấn luyện hết từ đầu, chỉ cần huấn luyện trong 1 vùng đã phân chia giữa các agent.


In [ ]:
#Tạo 1 bảng cho biết tổng số lượng sản phẩm đã được mua của từng sản phẩm
#print(transaction[['PRODUCT_ID','QUANTITY','SALES_VALUE']].groupby('PRODUCT_ID').sum().sort_values('QUANTITY',ascending=False))
#transaction[['DAY','QUANTITY']].groupby('DAY').sum()
print(transaction[transaction['PRODUCT_ID']==6534178])
print(product[product['PRODUCT_ID']==6534178])

